In [72]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pickle
import lightgbm as lgb
import sklearn
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import r2_score
import colorama

In [73]:
X_train= pickle.load(open('./数据集/data_train','rb'))
Y_train = X_train.pop('tradeMoney')
data_test = pickle.load(open('./数据集/data_test','rb'))
target_test = pd.read_csv('./数据集/评分文件/sub_a_913.csv')

In [74]:
print(len(X_train))
print(len(Y_train))

40134
40134


In [75]:
# N_HYPEROPT_PROBES = 500
# HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest

# # ----------------------------------------------------------

# colorama.init()

# # ---------------------------------------------------------------------
# def get_lgb_params(space):
#     lgb_params = dict()
#     lgb_params['boosting_type'] = space['boosting_type'] if 'boosting_type' in space else 'gbdt'
#     lgb_params['objective'] = 'regression'
#     lgb_params['metric'] = 'rmse'
#     lgb_params['learning_rate'] = space['learning_rate']
#     lgb_params['num_leaves'] = int(space['num_leaves'])
#     lgb_params['min_data_in_leaf'] = int(space['min_data_in_leaf'])
#     lgb_params['min_sum_hessian_in_leaf'] = space['min_sum_hessian_in_leaf']
#     lgb_params['max_depth'] = -1
#     lgb_params['lambda_l1'] = space['lambda_l1'] if 'lambda_l1' in space else 0.0
#     lgb_params['lambda_l2'] = space['lambda_l2'] if 'lambda_l2' in space else 0.0
#     lgb_params['max_bin'] = int(space['max_bin']) if 'max_bin' in space else 256
#     lgb_params['feature_fraction'] = space['feature_fraction']
#     lgb_params['bagging_fraction'] = space['bagging_fraction']
#     lgb_params['bagging_freq'] = int(space['bagging_freq']) if 'bagging_freq' in space else 1
#     lgb_params['nthread'] = 4
#     return lgb_params
# # ---------------------------------------------------------------------

# obj_call_count = 0
# cur_best_score = 0 # 0 or np.inf
# log_writer = open( './数据集/lgb-hyperopt-log.txt', 'w' )


# def objective(space):
#     global obj_call_count, cur_best_score

#     obj_call_count += 1

#     print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )

#     lgb_params = get_lgb_params(space)

#     sorted_params = sorted(space.items(), key=lambda z: z[0])
#     params_str = str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params])
#     print('Params: {}'.format(params_str) )
    
#     kf = KFold(n_splits=3, shuffle=True, random_state=0)
#     out_of_fold = np.zeros(len(X_train))
#     for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
#         D_train = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx])
#         D_val = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx])
#         # Train
#         num_round = 10000
#         clf = lgb.train(lgb_params,
#                            D_train,
#                            num_boost_round=num_round,
#                            # metrics='mlogloss',
#                            valid_sets=D_val,
#                            # valid_names='val',
#                            # fobj=None,
#                            # feval=None,
#                            # init_model=None,
#                            # feature_name='auto',
#                            # categorical_feature='auto',
#                            early_stopping_rounds=200,
#                            # evals_result=None,
#                            verbose_eval=False,
#                            # learning_rates=None,
#                            # keep_training_booster=False,
#                            # callbacks=None
#                            )
#         # predict
#         nb_trees = clf.best_iteration
#         val_loss = clf.best_score['valid_0']
#         print('nb_trees={} val_loss={}'.format(nb_trees, val_loss))
#         out_of_fold[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=nb_trees)
#         score = r2_score(out_of_fold, Y_train)

#     print('val_r2_score={}'.format(score))

#     log_writer.write('score={} Params:{} nb_trees={}\n'.format(score, params_str, nb_trees ))
#     log_writer.flush()

#     if score>cur_best_score:
#         cur_best_score = score
#         print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
#     return {'loss': -score, 'status': STATUS_OK}

# # --------------------------------------------------------------------------------

# space ={
#         'num_leaves': hp.quniform ('num_leaves', 10, 100, 1),
#         'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 1),
#         'feature_fraction': hp.uniform('feature_fraction', 0.75, 1.0),
#         'bagging_fraction': hp.uniform('bagging_fraction', 0.75, 1.0),
#         'learning_rate': hp.uniform('learning_rate', 0, 0.01),
# #         'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),
#         'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
#         'max_bin': hp.quniform ('max_bin', 88, 200, 1),
#         'bagging_freq': hp.quniform ('bagging_freq', 1, 15, 1),
#         'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
#         'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
#        }

# trials = Trials()
# best = hyperopt.fmin(fn=objective,
#                      space=space,
#                      algo=HYPEROPT_ALGO,
#                      max_evals=N_HYPEROPT_PROBES,
#                      trials=trials,
#                      verbose=1)

# print('-'*50)
# print('The best params:')
# print( best )
# print('\n\n')

In [79]:
######################### 用参考文件训练出来的参数预测

In [80]:
def get_model_type():   
    model = lgb.LGBMRegressor(
              bagging_fraction=0.9378794860721779 ,
            bagging_freq=3,
            feature_fraction=0.7652941762489258 ,
            lambda_l1=8.697270687215997 ,
            lambda_l2=4.1891718551655295 ,
            learning_rate=0.008101761507859587 ,
            max_bin=157 ,
            min_data_in_leaf=47 ,
            min_sum_hessian_in_leaf=1.638570722524837 ,
            num_leaves=30 ,
            nb_trees=8613
            )
    return model

In [81]:
model = get_model_type()
model.fit(X_train, Y_train)

LGBMRegressor(bagging_fraction=0.9378794860721779, bagging_freq=3,
              boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              feature_fraction=0.7652941762489258, importance_type='split',
              lambda_l1=8.697270687215997, lambda_l2=4.1891718551655295,
              learning_rate=0.008101761507859587, max_bin=157, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=47,
              min_split_gain=0.0, min_sum_hessian_in_leaf=1.638570722524837,
              n_estimators=100, n_jobs=-1, nb_trees=8613, num_leaves=30,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [82]:
y_pred = model.predict(data_test)
score = r2_score(y_pred, target_test)
print('val_r2_score是{}'.format(score))

val_r2_score是-0.23531879893550145


In [83]:
################################ 调用一些别的模型参数预测

In [84]:
def get_model_type():   
#     model = lgb.LGBMRegressor(
#               bagging_fraction=0.9378794860721779 ,
#             bagging_freq=3,
#             feature_fraction=0.7652941762489258 ,
#             lambda_l1=8.697270687215997 ,
#             lambda_l2=4.1891718551655295 ,
#             learning_rate=0.008101761507859587 ,
#             max_bin=157 ,
#             min_data_in_leaf=47 ,
#             min_sum_hessian_in_leaf=1.638570722524837 ,
#             num_leaves=30 ,
#             nb_trees=8613
#             )
    model = lgb.LGBMRegressor(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='mse',
            max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
            n_estimators=600, subsample=0.9, colsample_bytree=0.7)
    return model

In [85]:
model = get_model_type()
model.fit(X_train, Y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=5, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=600, n_jobs=-1, num_leaves=31, objective='mse',
              random_state=2019, reg_alpha=0.25, reg_lambda=0.25, silent=True,
              subsample=0.9, subsample_for_bin=200000, subsample_freq=0)

In [86]:
y_pred = model.predict(data_test)
score = r2_score(y_pred, target_test)
print('val_r2_score是{}'.format(score))

val_r2_score是0.9705222296078768
